In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import sys, os
sys.path.append('../')
from data_handling import Container,load_data,Normalize,Denormalize
from plot_style.style_prab import load_preset
from plot_style.style_prab import colors as colors_preset
from model import NN, Loss, L1Loss
load_preset(scale=1,font_path='../plot_style/font')

matplotlib preset loaded


In [2]:
run generate_data

In [3]:
path = './toydata.h5'
OD_models = [NN(2,1,hidden=2,layers=2),
            NN(1,1,hidden=2,layers=2)]

MSE_models = [NN(1,1,hidden=2,layers=2)]

trainset, testset, norm  = load_data(path,
                                ['x'],
                                ['y'],
                                samples=5000,
                                ratio=0.9,
                                random=False)

norm[0][0]=0
norm[0][1]=1
norm[1][0]=0
norm[1][1]=1

trainset, testset, _  = load_data(path,
                                ['x'],
                                ['y'],
                                samples=5000,
                                ratio=0.9,
                                norm=norm,
                                random=False)

    
trainset = Container(trainset)
testset = Container(testset)

train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=2**10,
    shuffle=True,
    num_workers=1)

test_loader = torch.utils.data.DataLoader(
    testset,
    batch_size=2**10,
    shuffle=True,
    num_workers=1)


In [4]:
epochs = 1000
models = OD_models 
opt1 = torch.optim.Adam(models[0].parameters(), lr=0.01)
opt2 = torch.optim.Adam(models[1].parameters(), lr=0.01)
criterion = Loss()

models[0].train(True)
models[1].train(True)
l_OD = []
for epoch in range(epochs):    
    if epoch >= 500:
        mloss = 0
        for data, target in train_loader:
            opt1.zero_grad()
            opt2.zero_grad()
            x_pred = models[0](torch.cat([data,target],1))
            y_pred = models[1](x_pred)
            loss = criterion(y_pred, target, x_pred, data, models[0].sigma, models[1].sigma)
            loss.backward()    
            opt1.step()
            opt2.step()
            mloss += loss
    else:
        mloss = 0
        for data, target in train_loader:
            opt2.zero_grad()
            y_pred = models[1](data)
            loss = criterion(y_pred, target, data, data, models[0].sigma, models[1].sigma)
            loss.backward()
            opt2.step()
            mloss += loss
 
            opt1.zero_grad()
            x_p = models[0](torch.cat([data,target],1))
            y_p = models[1](x_p)
            loss = criterion(y_p, target, x_p, data, models[0].sigma, models[1].sigma)
            loss.backward()
            opt1.step()
    if epoch%100 == 0:
        print(epoch)
    l_OD.append((mloss).detach())

0
100
200
300
400
500
600
700
800
900


In [ ]:
epochs = 500
models = MSE_models 
opt1 = torch.optim.Adam(models[0].parameters(), lr=0.01)
criterion = L1Loss()

models[0].train(True)
l_MSE = []
for epoch in range(epochs):    
    mloss = 0
    for data, target in train_loader:
        opt1.zero_grad()
        y_pred = models[0](data)
        loss = criterion(y_pred, target)
        loss.backward()
        opt1.step()
        mloss += loss
            
    l_MSE.append((mloss).detach())

In [ ]:
plt.plot(np.log(l_OD),label='OD')
plt.plot(np.log(l_MSE),label='MSE')
plt.legend()

In [ ]:
from scipy.odr import Model, Data, ODR

def tls_f(B, x):
    '''Linear function y = m*x + b'''
    # B is a vector of the parameters.
    # x is an array of the current x values.
    # x is in the same format as the x passed to Data or RealData.
    #
    # Return an array in the same format as y passed to Data or RealData.
    return B[0]*x**2 + B[1]*x + B[2]

linear = Model(tls_f)
mydata = Data(trainset.x.flatten(), trainset.y.flatten(), wd=1., we=1.)
myodr = ODR(mydata, linear, beta0 = [0., 0., 0.])
myoutput = myodr.run()
k_odr = myoutput.beta
myoutput.pprint()

In [ ]:
MSE_models[0].train(False)
OD_models[0].train(False)
OD_models[1].train(False)



fig,ax = plt.subplots(1,1,figsize=(2.5,2.2))
x = torch.linspace(-4,4).unsqueeze(1)
x_lin = np.linspace(-4,4)

ax.hist2d(trainset.x.detach().flatten().numpy(),trainset.y.detach().flatten().numpy(),cmap='bone_r',bins=25,vmax=80)
ax.plot(testset.x.detach().flatten().numpy()[::4],testset.y.detach().flatten().numpy()[::4],'.',color='k',ms=3.,mew=0,label='Data samples')

ax.plot(x_lin,f(x_lin),color='k',label=r'$f$_true',lw=1.)
ax.plot(x.detach().numpy(),OD_models[1](x).detach().numpy(),color=colors_preset[0],label='OD',lw=1.)
ax.plot(x.detach().numpy(),MSE_models[0](x).detach().numpy(),color=colors_preset[3],label='MSE',lw=1.)
k_ols = np.polyfit(trainset.x.flatten(),trainset.y.flatten(),2)

#ax.plot(x_lin,k_ols[0]*x_lin**2+k_ols[1]*x_lin+k_ols[2],'-o',color='gray',label='OLS',lw=0.3,ms=1.)
#ax.plot(x_lin,k_odr[0]*x_lin**2+k_odr[1]*x_lin+k_odr[2],'-s',color='gray',label='ODR',lw=0.3,ms=1)


ax.set_xlim(-4,4)
ax.set_ylim(-1.7,4.2)
ax.set_xlabel(r'$x$ (arb. units)')
ax.set_ylabel('$y$ (arb. units)')

ax.legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left",
          #mode="expand", 
          borderaxespad=0, 
          ncol=2,
          frameon=False)
plt.savefig('toy_problem.pdf')